In [ ]:
# PREAMBLE
import numpy as np
from matplotlib import pyplot as plt

from astropy.cosmology import Planck15 as cosmo
from astropy.table import Table
from astropy import units as u

import camb
from camb import model

import splashback.cosmic_noise as cn
import splashback.cluster as cl
cn = reload(cn)
cl = reload(cl)

***$C_\mathrm{LSS}$***
======

In this notebook the shear noise caused by uncorrelated LSS structure is computed.

***Power Spectrum by camb***

In [ ]:
#Straight from tutorial. 
pars = camb.CAMBparams()
pars.set_cosmology(H0=70, ombh2=0.024, omch2=0.124)
pars.set_dark_energy()
pars.InitPower.set_params(ns=0.965)
pars.NonLinear = camb.model.NonLinear_both

#Camb returns the power spectrum (in Mpc-3) as a function of k (in Mpc)
Pk = camb.get_matter_power_interpolator(pars, zmax=10)#, hubble_units=False, k_hunit=False)
Pk_2 = camb.get_matter_power_interpolator(pars, zmax=10, hubble_units=False, k_hunit=False)

plt.loglog(np.geomspace(1e-4, 1),np.exp(Pk(0, np.log(np.geomspace(1e-4, 1.))))[0], label='k unit' )
plt.loglog(np.geomspace(1e-4, 1),np.exp(Pk_2(0, np.log(np.geomspace(1e-4, 1.))))[0], label='no k unit')

plt.legend()
plt.show()

results = camb.get_results(pars)
s8 = np.array(results.get_sigma8())
print "s8:", s8

***Compute an example Power spectrum***

In [ ]:
p_k_hist, W_hist = cn.P_k_gen(z_list =Table.read('data/photoz/COSMOS2015.fits')['z'], verbose=True)
p_k_func, W_func = cn.P_k_gen(verbose=True)
p_k_plane, W_plane = cn.P_k_gen(z_s=0.7, verbose=True)

print "COSMOS2015"

ls = np.geomspace(20, 1e4)
plt.figure()
plt.loglog(ls, p_k_hist(ls)*ls*(ls+1)/2/np.pi, label='histogram')
plt.loglog(ls, p_k_func(ls)*ls*(ls+1)/2/np.pi, label='function')
plt.loglog(ls, p_k_plane(ls)*ls*(ls+1)/2/np.pi, label='single plane')
plt.gca().set_xlabel('Matter power spectrum P(k) (Mpc-3)')
plt.gca().set_ylabel('k (Mpc-1)')
plt.legend()
plt.show()

plt.figure()
plt.plot(p_k_hist(ls)/p_k_func(ls))
plt.title("ratio histogram/function")
plt.gca().set_xlabel('ratio')
plt.gca().set_ylabel('k (Mpc-1)')
plt.show()


ws = np.linspace(0, 9000)
plt.figure()
plt.loglog(ws, W_hist(ws), label='histogram')
plt.loglog(ws, W_func(ws), label='function')
plt.loglog(ws, W_plane(ws), label='single plane')
plt.gca().set_xlabel('Comoving distance (Mpc)')
plt.gca().set_ylabel('Lensing ratio W(w)')
plt.legend()
plt.show()

***CREATE SHEAR COVARIANCE MATRIX***

In [ ]:
bin_edges = np.geomspace(.2, 9, 11)*u.Mpc
data_CCCP = Table.read('data/CCCPnomergers.fits')

for i in xrange(len(data_CCCP)):
    print data_CCCP['name'][i]
    sky_edges = (bin_edges/data_CCCP['da'].quantity[i]).to('arcmin')
    table = Table.read('data/photoz/individual/'+data_CCCP['name'][i]+'.fits')
    p_k_hist = cn.P_k_gen(z_list=table['z'], weights=table['w'])
    print "P_k done"
    np.save("output/lss_covariance/"+data_CCCP['name'][i]+".npy", cn.CLSS(sky_edges, p_k_hist))
    print "saved"